## Добрый день если вы это читаете значит я не успел сделать третью лабу до того как вы её проверите.

In [1]:
# считывает MovieLens
from os import path
import pandas as pd

data_dir = "./ml-latest-small"

def read_csv(filename: str):
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")
tags = read_csv("tags")

In [2]:

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [78]:
movies.describe()

,movieId
count,9125.000000
mean,31123.291836
std,40782.633604
min,1.000000
25%,2850.000000
50%,6290.000000
75%,56274.000000
max,164979.000000


In [5]:
tags.describe()

,userId,movieId,timestamp
count,1296.000000,1296.000000,1.296000e+03
mean,417.026235,42278.949846,1.324337e+09
std,142.183440,44628.345568,1.093886e+08
min,15.000000,1.000000,1.137217e+09
25%,346.000000,2988.000000,1.243455e+09
50%,431.000000,26958.500000,1.342849e+09
75%,547.000000,72268.250000,1.440380e+09
max,663.000000,164979.000000,1.476651e+09


In [90]:
mov_tag = pd.merge(movies, tags, how = 'left')

In [91]:
mov_tag.describe()

,movieId,userId,timestamp
count,9732.000000,1296.000000,1.296000e+03
mean,31671.091759,417.026235,1.324337e+09
std,40931.884839,142.183440,1.093886e+08
min,1.000000,15.000000,1.137217e+09
25%,2828.750000,346.000000,1.243455e+09
50%,6371.500000,431.000000,1.342849e+09
75%,58426.750000,547.000000,1.440380e+09
max,164979.000000,663.000000,1.476651e+09


In [101]:
mov_tag.head()

,movieId,title,genres,userId,tag,timestamp,movie_id,tag_id
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,501,Pixar,1.29296e+09,0,108
1,2,Jumanji (1995),Adventure|Children|Fantasy,None,None,None,1,97
2,3,Grumpier Old Men (1995),Comedy|Romance,None,None,None,2,97
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,None,None,None,3,97
4,5,Father of the Bride Part II (1995),Comedy,431,steve martin,1.14046e+09,4,504


In [98]:
# заполняем пустые значения
mov_tag.fillna("None", inplace=True)


In [100]:
# перекодируем id с пробелами в плотные
mov_tag["movie_id"] = mov_tag["movieId"].astype("category").cat.codes.copy()
mov_tag["tag_id"] = mov_tag["tag"].astype("category").cat.codes.copy()

In [21]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [22]:
from scipy.sparse import csr_matrix
import numpy as np


In [102]:
last_movie_id = mov_tag["movie_id"].max()
last_tag_id = mov_tag["movie_id"].max()

In [103]:
# переводим матрицу взаимодействий фильмов и тегов к разреженному виду
from scipy.sparse import csr_matrix
import numpy as np
movie_x_tag = mov_tag[["movie_id", "tag_id"]].as_matrix()
movie_x_tag


array([[   0,  108],
       [   1,   97],
       [   2,   97],
       ..., 
       [9122,  551],
       [9123,  536],
       [9124,  536]], dtype=int16)

In [104]:
data = []
for i in mov_tag['tag']:
    data.append(1)


In [105]:
from scipy.sparse import coo_matrix
import numpy as np
movie_tag_matrix = coo_matrix((
    data,
    (
        mov_tag["movie_id"],
        mov_tag["tag_id"]
    )
))

In [106]:
test_indices = np.random.choice(
    ratings.index.values,
    replace=False,
    size=int(len(ratings.index.values) * 0.2)
)
test_data = ratings.iloc[test_indices]
train_data = ratings.drop(test_indices)

In [107]:
train_data.shape

(80004, 4)

In [108]:
sparse_info(movie_tag_matrix)

Размерности матрицы: (9125, 583)
Ненулевых элементов в матрице: 9732
Доля ненулевых элементов: 0.0018293662915012101
Среднее значение ненулевых элементов: 1.0
Максимальное значение ненулевых элементов: 1
Минимальное значение ненулевых элементов: 1


In [109]:
# train the model on a sparse matrix of item/user/confidence weights
import implicit
import time
start = time.time()
model = implicit.als.AlternatingLeastSquares(factors=50, iterations=50, regularization=0.01)
print("строим матрицу схожести по ALS")
model.fit(movie_tag_matrix)
print("построили матрицу схожести по ALS за {} секунд".format(
        time.time() - start))

строим матрицу схожести по ALS
построили матрицу схожести по ALS за 2.1915478706359863 секунд


In [110]:
test_data.head()

,userId,movieId,rating,timestamp
86225,577,1097,5.0,1111475317
62000,452,2527,3.0,976424003
71180,500,595,3.0,1228920972
91083,605,1938,3.0,980174106
74950,520,912,5.0,1142024691


In [117]:
sparse_info(movie_tag_matrix)

Размерности матрицы: (9125, 583)
Ненулевых элементов в матрице: 9708
Доля ненулевых элементов: 0.0018248549073051528
Среднее значение ненулевых элементов: 1.0024721878862795
Максимальное значение ненулевых элементов: 4
Минимальное значение ненулевых элементов: 1


In [124]:
pre = []
print("получаем рекомендации для всех фильмов")
start = time.time()
movie_matrix = movie_tag_matrix.tocsr()
for mov_id in mov_tag["movie_id"]:
    for movie_id_rec, score in model.recommend(mov_id, movie_matrix):
         pre.append((mov_id, movie_id_rec, score))
print("получили рекомендации для всех фильмов за {} секнуд".format(
        time.time() - start))

получаем рекомендации для всех фильмов


IndexError: index 583 is out of bounds for axis 0 with size 583

In [ ]:
# разобьём наблюдения на тестовую и обучающую выборки
np.random.seed(0)
test_indices = np.random.choice(
    range(user_item_matrix.nnz),
    replace=False,
    size=int(user_item_matrix.nnz * 0.2)
).tolist()
train_data = user_item_matrix.copy()
train_data.data[test_indices] = 0
train_data.eliminate_zeros()
print("размер обучающей выборки: {}".format(train_data.nnz))
test_data = user_item_matrix.copy()
test_data.data[:] = 0
test_data.data[test_indices] = user_item_matrix.data[test_indices]
test_data.eliminate_zeros()
print("размер тестовой выборки: {}".format(test_data.nnz))

In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, reciprocal_rank

# обучаем абы какую модель
model = LightFM(loss="bpr")
model.fit(
    train_data,
    num_threads=4
)

train_mrr = reciprocal_rank(model, train_data).mean()
test_mrr = reciprocal_rank(model, test_data).mean()
print('MRR: train %.2f, test %.2f.' % (train_mrr, test_mrr))
for k in [5, 10, 15, 20]:
    train_precision = precision_at_k(model, train_data, k=k).mean()
    test_precision = precision_at_k(model, test_data, k=k).mean()
    print('Precision@%d: train %.2f, test %.2f.' % (k, train_precision, test_precision))

In [ ]:
# выберем информацию по жанрам фильмов
movies_genres = ratings[["movieId", "movie_id"]].drop_duplicates().join(
    movies,
    on="movieId",
    rsuffix="codes",
    lsuffix="movies",
    sort=True
).fillna("None")[["movie_id", "genres"]]
movies_genres["genres_set"] = movies_genres["genres"].apply(lambda x: set(x.split("|")))
movies_genres.head()

In [ ]:
# все возможные жанры
from functools import reduce

reduce(lambda acc, ele: acc.union(ele), movies_genres["genres_set"].tolist(), set())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# приравняем None и (no genres listed)
movies_genres.loc[movies_genres["genres"] == "(no genres listed)", "genres"] = "None"
# уберём все спецсимволы, кроме |
movies_genres["genres"] = movies_genres["genres"].apply(
    lambda x: x.replace("-", "")
)

movies_features = CountVectorizer().fit_transform(movies_genres["genres"])
movies_features

In [ ]:
print(movies_genres["genres"][:1])
print(movies_features[0].todense())

In [ ]:
# добавляем к movie_id ещё и информацию о жанрах
from scipy.sparse import hstack, identity

features = hstack([
    identity(movies_genres.shape[0]),
    movies_features
])

In [ ]:
features

In [ ]:
# обучаем модель с жанрами
hybrid = LightFM(loss="bpr")
hybrid.fit(
    train_data,
    num_threads=4,
    item_features=features
)

train_mrr = auc_score(hybrid, train_data, item_features=features).mean()
test_mrr = auc_score(hybrid, test_data, item_features=features).mean()
print('MRR: train %.2f, test %.2f.' % (train_mrr, test_mrr))
#for k in [5, 10, 15, 20]:
    #train_precision = precision_at_k(hybrid, train_data, item_features=features, k=k).mean()
    #test_precision = precision_at_k(hybrid, test_data, item_features=features, k=k).mean()
#print('Precision@%d: train %.2f, test %.2f.' % (k, train_precision, test_precision))